Intro :
In my learning journey to become a future data scientist and machine learning expert , I have decided to implement some of the useful linear algebra technics in the nowadays widely used coding language in data science and machine learning : Python . This is why I have decided to start with the Gaussian Elimination , one of the fundamental algorithms in Linear Algebra :

In [ ]:
import numpy as np

0/ Auxiliary functions : those are some row operations that might be helpful to use later in our job on matrices : 

In [ ]:
def swap_rows(M, row_index_1, row_index_2):
    """
    Swap rows in the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to perform row swaps on.
    - row_index_1 (int): Index of the first row to be swapped.
    - row_index_2 (int): Index of the second row to be swapped.
    """
    M[[row_index_1,row_index_2]]=M[[row_index_2,row_index_1]]
    return M

def get_index_first_non_zero_value_from_column(M, column, starting_row):
    """
    Retrieve the index of the first non-zero value in a specified column of the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to search for non-zero values.
    - column (int): The index of the column to search.
    - starting_row (int): The starting row index for the search.

    Returns:
    int: The index of the first non-zero value in the specified column, starting from the given row.
                Returns -1 if no non-zero value is found.
    """
    i=starting_row+1
    j=column
    n=M.shape[0]
    while ((i!=n)and(M[i,j]==0)):
        i=i+1
    if (i==n) :
        return -1
    if (M[i,j]!=0):
        return i
    else :
        return -1

def get_index_first_non_zero_value_from_row(M, row, augmented = False):
    """
    Find the index of the first non-zero value in the specified row of the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to search for non-zero values.
    - row (int): The index of the row to search.
    - augmented (bool): Pass this True if you are dealing with an augmented matrix, 
                        so it will ignore the constant values (the last column in the augmented matrix).

    Returns:
    int: The index of the first non-zero value in the specified row.
                Returns -1 if no non-zero value is found.
    """
    p=M.shape[1]
    if augmented :
        p=p-1
    i=row
    j=0
    while ((j!=p)and(M[i,j]==0)):
        j=j+1
    if (j==p) :
        return -1
    if (M[i,j]!=0):
        return j
    else :
        return -1

def augmented_matrix(A, B):
    """
    Create an augmented matrix by horizontally stacking two matrices A and B.

    Parameters:
    - A (numpy.array): First matrix.
    - B (numpy.array): Second matrix.

    Returns:
    - numpy.array: Augmented matrix obtained by horizontally stacking A and B.
    """
    M=np.hstack((A,B))
    return M

def null_matrix(M):
    return not M.any()

def has_all_zero_row(M):
    for row in M:
        if np.all(row == 0):
            return True
    return False

1/ Row Echelon Form :
In this part of the code , we will try turning our matrix into row echelon form . Row echelon form is a matrix nxp that has all its a_(i,j) elements 0 for 1<=i<=n and 1<=j<=p where i<j :

In [ ]:
def is_row_echelon(M):
    n, p = M.shape
    for i in range(n):
        non_zero_found = False
        for j in range(p):
            if M[i, j] != 0:
                if non_zero_found:
                    return False
                non_zero_found = True
                if any(M[k, j] != 0 for k in range(i)):
                    return False
            elif non_zero_found:
                break
    return True

def row_echelon_form(A, B):
    M = np.hstack((A, B.reshape(-1, 1)))  # Create the augmented matrix
    if null_matrix(M):
        print("Singular system.")
        return M
    if is_row_echelon(M):
        print("Already Row Echelon Form")
        return M

    n, p = M.shape
    for i in range(n):
        # If the pivot is zero, swap with a row below that has a non-zero element in the same column
        if M[i, i] == 0:
            for j in range(i + 1, n):
                if M[j, i] != 0:
                    M = swap_rows(M, i, j)
                    break
        # If the entire column is zero, check for singularity
        if M[i, i] == 0:
            if has_all_zero_row(M):
                print("Singular system.")
                return M
            continue
        pivot = M[i, i]
        M[i] = M[i] / pivot
        for j in range(i + 1, n):
            M[j] = M[j] - M[j, i] * M[i]
    print("Done")
    return M

2/Back substitution :
It is almost the final step in the algorithm since it will turn our row echelon form matrix into a reduced row echelon form matrix

In [ ]:
def back_substitution(M):
    """ Perform back substitution on an augmented matrix in reduced row echelon form. """

    # Make a copy of the input matrix to avoid modifying the original
    M = M.copy()

    # Get the number of rows (and columns) in the matrix of coefficients
    num_rows = M.shape[0]

    # Iterate from bottom to top
    for row in reversed(range(num_rows)):
        # substitution_row is the current row in the iteration
        substitution_row = M[row, :]

        # Get the index of the first non-zero element in the substitution row. 
        index = np.argmax(substitution_row[:-1] != 0)

        # Iterate over the rows above the substitution_row
        for j in range(row):
            # Get the row to be reduced. 
            row_to_reduce = M[j, :]

            # Get the value of the element at the found index in the row to reduce
            value = row_to_reduce[index]

            # Perform the back substitution step using the formula row_to_reduce -> row_to_reduce - value * substitution_row
            row_to_reduce = row_to_reduce - value * substitution_row

            # Replace the updated row in the matrix
            M[j, :] = row_to_reduce

    # Extract the solution from the last column
    solution = M[:, -1]

    return solution

3/Summing up :
Finally , we have summed up the 2 functions into one to be able to get the final gaussian elimination ;

In [ ]:
def gaussian_elimination(A, B):
    """
    Solve a linear system represented by an augmented matrix using the Gaussian elimination method.

    Parameters:
    - A (numpy.array): Square matrix of size n x n representing the coefficients of the linear system
    - B (numpy.array): Column matrix of size 1 x n representing the constant terms.

    Returns:
    numpy.array or str: The solution vector if a unique solution exists, or a string indicating the type of solution.
    """

    # Get the matrix in row echelon form
    row_echelon_M = row_echelon_form(A,B)

    # If the system is non-singular, then perform back substitution to get the result.
    if np.linalg.det(row_echelon_M!=0):
        solution = back_substitution(row_echelon_M)
    else:
        return row_echelon_M  # Return the string indicating the type of solution (e.g., "No solution")

    return solution
        